In [1]:
import tensorflow as tf 
import tensorflow_datasets as tfds
import numpy as np 
import matplotlib.pyplot as plt


In [3]:
dataset, dataset_info = tfds.load("emnist", split=["test", "train"], shuffle_files=True, as_supervised=True, with_info=True)